In [ ]:
from graph_gaussian_process.examples import compile, sample_kwargs_from_env
from graph_gaussian_process.kernels import ExpQuadKernel
from graph_gaussian_process.util import plot_band, spatial_neighborhoods, neighborhood_to_edge_index
import matplotlib as mpl
from matplotlib import pyplot as plt
import numpy as np

mpl.rcParams["figure.dpi"] = 144

In [ ]:
# Set up the parameters of the model.
num_obs = 100
alpha = 1.2
rho = 0.1
mu = 1
eps = 1e-3
np.random.seed(1)

# Simulate the process and observations.
x = np.linspace(0, 1, num_obs)
X = x[:, None]
kernel = ExpQuadKernel(alpha, rho, eps)
cov = kernel(X)
eta = np.random.multivariate_normal(mu * np.ones(num_obs), cov)
lam = np.exp(eta)
y = np.random.poisson(lam)

# Assemble the state into a dictionary.
state = {
    "num_obs": num_obs,
    "alpha": alpha,
    "rho": rho,
    "eps": eps,
    "x": x,
    "X": X,
    "eta": eta,
    "lam": lam,
    "y": y,
    "num_dims": 1,
}

In [ ]:
def plot_state(state, ax=None):
    ax = ax or plt.gca()
    ax.plot(state["x"], state["lam"], label=r"rate $\lambda$")
    ax.scatter(state["x"], state["y"], marker=".", color="black", label=r"counts $y$")
    ax.set_xlabel(r"Coordinate $x$")
    ax.legend()

plot_state(state)

In [ ]:
model = compile("poisson-regression.stan")

In [ ]:
num_neighbors = 5
neighborhoods = spatial_neighborhoods((num_obs,), num_neighbors)
edge_index = neighborhood_to_edge_index(neighborhoods)
connectivity = {"edge_index": edge_index + 1, "num_edges": edge_index.shape[1]}

fit = model.sample(state | connectivity, **sample_kwargs_from_env())

In [ ]:
plot_state(state)
plot_band(x, np.exp(fit.stan_variable("eta")), label="inferred rate")